In [ ]:
from google.colab import drive
drive.mount('/content/drive') # , force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd 
import numpy as np
import re
import os
import ast
from glob import glob
from tqdm import tqdm
tqdm.pandas()

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 200)

def flatten_list(l):
  return [item for sublist in l for item in sublist]

from collections import Counter

In [ ]:
metadata_df = pd.read_csv('/content/drive/MyDrive/metavis/socrata_nycopendata_metadata_fetched_20211105.csv')
metadata_df['columns'] = metadata_df['columns'].apply(ast.literal_eval)
col_metadata_df = metadata_df.explode('columns').reset_index(drop=True)
col_metadata_df = col_metadata_df.dropna(subset=['columns']).reset_index(drop=True)
col_metadata_df = pd.concat([col_metadata_df['data_uuid'], pd.json_normalize(col_metadata_df['columns'])], axis=1)

col_metadata_df = col_metadata_df[ ['data_uuid', 'id', 'tableColumnId', 'name', 'fieldName', 'dataTypeName', 'description'] + [col for col in col_metadata_df.columns if col.startswith('cachedContents.')] ]
col_metadata_df['name'] = col_metadata_df['name'].apply(lambda x: x.replace('_',' ')).replace('',np.nan)
col_metadata_df['description'] = col_metadata_df['description'].replace('',np.nan)
col_metadata_df['description'] = col_metadata_df['name'].fillna('') + col_metadata_df['description'].fillna('').apply(lambda x: ' <SEP> ' + x if len(x)>0 else x)

In [ ]:
tabular_data_uuid_list = metadata_df.query('assetType == "dataset" & viewType == "tabular"')['data_uuid']
tabular_dataset_col_metadata_df = col_metadata_df[col_metadata_df['data_uuid'].isin(tabular_data_uuid_list)]
data_uuid_to_name_mapping = metadata_df.set_index('data_uuid')['name'].to_dict()
tabular_dataset_col_metadata_df['data_name'] = tabular_dataset_col_metadata_df['data_uuid'].map(data_uuid_to_name_mapping)
tabular_dataset_col_metadata_df = tabular_dataset_col_metadata_df[['data_uuid', 'data_name', 'id', 'tableColumnId', 'name', 'fieldName', 'dataTypeName', 
       'description', 'cachedContents.largest', 'cachedContents.non_null', 'cachedContents.null', 'cachedContents.not_null',
       'cachedContents.top', 'cachedContents.smallest', 'cachedContents.count', 'cachedContents.cardinality', 'cachedContents.average',
       'cachedContents.sum', 'cachedContents.largest.url', 'cachedContents.smallest.url', 'cachedContents.largest.description', 'cachedContents.smallest.description']]
tabular_dataset_col_metadata_df.to_csv('/content/drive/MyDrive/metavis/tabular_dataset_col_metadata_df.csv',index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/metavis/tabular_dataset_col_metadata_df.csv')

In [ ]:
df['name_cleaned'] = df['name'].apply(lambda x: re.sub('[^a-z0-9 ]','',x.lower().strip()).strip())

In [ ]:
df.name_cleaned.value_counts()[df.name_cleaned.value_counts()>1][:30]

borough                   642
latitude                  413
longitude                 411
dbn                       392
bin                       366
year                      356
community board           352
nta                       346
bbl                       346
census tract              337
council district          326
postcode                  300
school name               282
category                  246
grade                     184
zip codes                 180
community districts       180
city council districts    179
police precincts          179
borough boundaries        178
district                  166
agency                    165
number tested             162
mean scale score          160
city                      154
program type              144
level 3                   136
level 34                  136
level 4                   136
level 1                   135
Name: name_cleaned, dtype: int64

### BERT

In [ ]:
!pip install transformers --quiet

import torch
from torch import nn, optim, Tensor
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def encode(sentences, model, tokenizer, device):
    model.eval()
    model.to(device)
    features = tokenizer(sentences,return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**features)
        last_hidden_states = outputs[0]
    embedding = torch.mean(last_hidden_states, dim = 1).squeeze()
    return  embedding

# col_metadata_df['description'] = col_metadata_df['description'].apply(lambda x: x.lower())
# col_metadata_df['embedding'] = col_metadata_df['description'].progress_apply(lambda x: encode(x, model, tokenizer, device))

### GloVe

In [ ]:
import gensim
import gensim.downloader as api
glove_model = api.load('glove-wiki-gigaword-100') # glove-100dimension is 128 MB

def remove_domain_frequent_words(input, domain_frequent_words = {}): 
  output = list(set(input).difference(domain_frequent_words))
  return output

def get_nan_embedding(wv_model):
  na_array = np.empty((wv_model.vector_size,))
  na_array[:] = np.nan
  return na_array

def get_embedding(token, wv_model):
  try:
    return wv_model.wv[token]
  except:
    return get_nan_embedding(wv_model)

def create_embedding(data, col, domain_frequent_words = {}):
  data['_tokens'] = data[col].fillna('').apply(gensim.parsing.preprocessing.remove_stopwords).apply(gensim.utils.simple_preprocess)
  data['_tokens_wo_domain_freq'] = data['_tokens'].apply(lambda li: remove_domain_frequent_words(li, domain_frequent_words = domain_frequent_words))
  data[col+'__glove_word_embedding'] = data['_tokens_wo_domain_freq'].apply(lambda li: np.nanmean([get_embedding(x, wv_model = glove_model) for x in li], axis=0) if len(li)>0 else get_nan_embedding(wv_model = glove_model) )
  data = data.drop(['_tokens','_tokens_wo_domain_freq'],axis=1)
  return data


In [ ]:
col_metadata_df = create_embedding(col_metadata_df, 'name', {'nyc', 'data', '<SEP>'})
col_metadata_df = create_embedding(col_metadata_df, 'description', {'nyc', 'data', '<SEP>'})
col_name_based_word_embedding_for_datasets = col_metadata_df.groupby('data_uuid')['name__glove_word_embedding'].apply(list).apply(lambda li: np.nanmean(li, axis=0)).reset_index().rename(columns = {'name__glove_word_embedding':'colnames__glove_word_embedding'})
col_name_based_word_embedding_for_datasets['colnames__glove_word_embedding'] = col_name_based_word_embedding_for_datasets['colnames__glove_word_embedding'].apply(lambda arr: np.nan if all(np.isnan(arr)) else arr)


name_and_description_df = metadata_df[['data_uuid','name','description']].copy()
name_and_description_df = create_embedding(name_and_description_df, 'name', {'nyc', 'data'})
name_and_description_df = create_embedding(name_and_description_df, 'description', {'nyc', 'data'})
name_and_description_df = name_and_description_df.rename(columns = {'name__glove_word_embedding':'dataset_name__glove_word_embedding'})
name_and_description_df = name_and_description_df.rename(columns = {'description__glove_word_embedding':'dataset_desc__glove_word_embedding'})

name_and_description_df['dataset_name__glove_word_embedding'] = name_and_description_df['dataset_name__glove_word_embedding'].apply(lambda arr: np.nan if all(np.isnan(arr)) else arr)
name_and_description_df['dataset_desc__glove_word_embedding'] = name_and_description_df['dataset_desc__glove_word_embedding'].apply(lambda arr: np.nan if all(np.isnan(arr)) else arr)



dataset_embedding_df = pd.merge(name_and_description_df,  col_name_based_word_embedding_for_datasets, how = 'left')

dataset_embedding_df['dataset_embedding'] = dataset_embedding_df[['colnames__glove_word_embedding','dataset_name__glove_word_embedding','dataset_desc__glove_word_embedding']].apply(lambda row: row['colnames__glove_word_embedding'] if not isinstance(row['colnames__glove_word_embedding'],float) else row['dataset_name__glove_word_embedding'] if not isinstance(row['dataset_name__glove_word_embedding'], float) else row['dataset_desc__glove_word_embedding'] ,axis=1)
assert(dataset_embedding_df['dataset_embedding'].isnull().mean()==0)

dataset_embedding_df[['data_uuid','dataset_embedding']].to_csv(X'/content/drive/MyDrive/metavis/dataset_embedding_v20211117.csv',index=False)

### Read Back and Evaluate Similarity

In [ ]:
import ast
import numpy as np
def from_np_array(array_string):
  # https://stackoverflow.com/a/42756309
  array_string = ','.join(array_string.replace('[ ', '[').split())
  return np.array(ast.literal_eval(array_string))
test = pd.read_csv('/content/drive/MyDrive/metavis/dataset_embedding_v20211117.csv', converters={'dataset_embedding': from_np_array})

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarities = cosine_similarity(col_metadata_df['embedding'].values)